# Relation Data EDA

This notebook computes summary statistics and performs sanity checks on the relation data processing pipeline (raw_data -> relations database and sentences tagged with key terms).

In [1]:
import pandas as pd
import pickle
import json
from collections import defaultdict
import spacy
import stanfordnlp
import pandas as pd
from spacy_stanfordnlp import StanfordNLPLanguage
import warnings
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from ipywidgets import interact

# nlp preprocessing pipeline
warnings.filterwarnings('ignore')
snlp = stanfordnlp.Pipeline(lang="en")
nlp = StanfordNLPLanguage(snlp)

# fix for importing utils
module_path = os.path.abspath(os.path.join('../data_processing'))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_processing_utils import read_spacy_docs, tag_terms

data_dir = '../data/relation_extraction'

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/mattboggess/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/mattboggess/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/Users/mattboggess/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/mattboggess/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/mattboggess/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/Users/mattboggess/sta

# What concepts actually match the text?


In [2]:
bio_sentences = read_spacy_docs("../data/preprocessed_data/Biology_2e_sentences_spacy", nlp)
bio_sentences += read_spacy_docs("../data/preprocessed_data/Life_Biology_sentences_spacy", nlp)
kb_terms = read_spacy_docs("../data/preprocessed_data/Life_Biology_kb_key_terms_spacy", nlp)

In [4]:
from collections import Counter

# count occurrences of all terms
term_counts = Counter()
for i, sentence in enumerate(bio_sentences):
    if i % 500 == 0:
        print(f"tagging sentence {i}/{len(bio_sentences)}")
    term_info = tag_terms(sentence, kb_terms, nlp)["found_terms"]
    term_counts.update({t: len(term_info[t]["indices"]) for t in term_info})

tagging sentence 0/44162
tagging sentence 500/44162
tagging sentence 1000/44162
tagging sentence 1500/44162
tagging sentence 2000/44162
tagging sentence 2500/44162
tagging sentence 3000/44162
tagging sentence 3500/44162
tagging sentence 4000/44162
tagging sentence 4500/44162
tagging sentence 5000/44162
tagging sentence 5500/44162
tagging sentence 6000/44162
tagging sentence 6500/44162
tagging sentence 7000/44162
tagging sentence 7500/44162
tagging sentence 8000/44162
tagging sentence 8500/44162
tagging sentence 9000/44162
tagging sentence 9500/44162
tagging sentence 10000/44162
tagging sentence 10500/44162
tagging sentence 11000/44162
tagging sentence 11500/44162
tagging sentence 12000/44162
tagging sentence 12500/44162
tagging sentence 13000/44162
tagging sentence 13500/44162
tagging sentence 14000/44162
tagging sentence 14500/44162
tagging sentence 15000/44162
tagging sentence 15500/44162
tagging sentence 16000/44162
tagging sentence 16500/44162
tagging sentence 17000/44162
tagging s

In [5]:
with open("../data/preprocessed_data/Life_Biology_kb_lexicon.json", "r") as f:
    lexicon = json.load(f)

tagged_concepts = []
non_tagged_concepts = []
for concept in lexicon.keys():
    if any([lemma in set(term_counts.keys()) for lemma in lexicon[concept]["lemma_representations"]]) or \
       any([text in set(term_counts.keys()) for text in lexicon[concept]["text_representations"]]):
        tagged_concepts.append({concept: lexicon[concept]})
    else:
        non_tagged_concepts.append({concept: lexicon[concept]})
print(f"{len(tagged_concepts)}/{len(non_tagged_concepts) + len(tagged_concepts)} bio kb concepts tagged in text")

df = {"concept": [], "text": []}
for concept in non_tagged_concepts:
    c = list(concept.keys())[0]
    df["concept"].append(c)
    df["text"].append(concept[c]["text_representations"])
pd.DataFrame(df).to_excel("../data/relation_extraction/diagnostics/not_tagged_concepts.xlsx", index=False)

3922/5686 bio kb concepts tagged in text


FileNotFoundError: [Errno 2] No such file or directory: '../data/relation_extraction/diagnostics/not_tagged_concepts.xlsx'

# Relations Database EDA

In [6]:
with open("../data/relation_extraction/relations_db.json", "r") as f:
    rdb = json.load(f)

## Summary Statistics


In [7]:
long_df = {"relation": [], "term_pair": [], "count_sentences": [], "found_sentence": []}
for relation in rdb:
    for term_pair in rdb[relation]: 
        long_df["relation"].append(relation)
        long_df["term_pair"].append(term_pair)
        long_df["count_sentences"].append(len(rdb[relation][term_pair]["sentences"]))
        long_df["found_sentence"].append(len(rdb[relation][term_pair]["sentences"]) > 0)
long_df = pd.DataFrame(long_df)

In [8]:
summary_df = long_df.groupby(["relation", "found_sentence"]).agg({"term_pair": "count",
                                                                  "count_sentences": ["sum", "mean"]})
summary_df.to_csv("../data/relation_extraction/summary/relation_counts_summary.csv", index=False)
summary_df

term_pair count_sentences          
                                count             sum      mean
relation     found_sentence                                    
abuts        False                132               0  0.000000
             True                  34             252  7.411765
agent        False               1630               0  0.000000
             True                 187             878  4.695187
angle        False                  1               0  0.000000
...                               ...             ...       ...
surface-area False                  6               0  0.000000
thickness    False                 19               0  0.000000
volume       False                108               0  0.000000
wavelength   False                 18               0  0.000000
width        False                  7               0  0.000000

[97 rows x 3 columns]

## Sentence Count Distributions

In [48]:
from ipywidgets import interact

def plot_sentence_counts(relation):
    
    df = long_df.query(f"count_sentences > 0 & relation == '{relation}'")
    df = df.query("count_sentences < 30")
    plt.hist(df.count_sentences, bins=30)
    plt.show()
    
interact(plot_sentence_counts, relation=list(rdb.keys()));

interactive(children=(Dropdown(description='relation', options=('no-relation', 'subclass-of', 'has-part', 'pos…

## Multi-Label Term Pairs

In [9]:
term_relation_mapping = {}
for relation in rdb:
    if relation == "no-relation":
        continue
    for tp in rdb[relation]:
        cp = rdb[relation][tp]["concept_pair"]
        pair = ", ".join([tp, cp])
        if pair in term_relation_mapping:
            term_relation_mapping[pair].append(relation)
        else:
            term_relation_mapping[pair] = [relation]
multi_label = {k:r for k, r in term_relation_mapping.items() if len(r) > 1}
print(f"Found {len(multi_label)} term pairs that have multiple relations")
with open(f"../data/relation_extraction/summary/multi_label_term_pairs.json", "w") as f:
    json.dump(multi_label, f, indent=4)

Found 2593 term pairs that have multiple relations


# Sanity Check Relation Extraction 

- How many word-pairs match the text on regex, but we don't match in the pipeline?
- How many word-pairs don't match sentences? Do these seem reasonable?
- Word-pairs with too many matches?

In [10]:
bio_sentences = read_spacy_docs("../data/preprocessed_data/Biology_2e_sentences_spacy", nlp)
bio_sentences += read_spacy_docs("../data/preprocessed_data/Life_Biology_sentences_spacy", nlp)

In [11]:
found_concepts = set()
df = {"sentences": [], 
      "relation": [], 
      "term-pair": [], 
      "concept-pair": [], 
      "tagged": [], 
      "synonym-tagged": [], 
      "count_sentences": [], 
      "textbook_match": [],
      "term1_found": [],
      "term2_found": []} 

for relation in rdb:
    print(relation)
    if relation == "no-relation":
        continue
    for i, term_pair in enumerate(rdb[relation]):
        if i % 100 == 0:
            print(f"Processing term pair {i}/{len(rdb[relation])}")
        
        sentences = rdb[relation][term_pair]["sentences"]
        count_sentences = len(sentences)
        df["relation"].append(relation)
        df["term-pair"].append(term_pair)
        concept_pair = rdb[relation][term_pair]["concept_pair"]
        df["concept-pair"].append(concept_pair)
        
        if count_sentences:
            df["tagged"].append(True)
            df["textbook_match"].append(True)
            df["term1_found"].append(True)
            df["term2_found"].append(True)
            found_concepts.add(concept_pair)
        else:
            terms = term_pair.split(" -> ")
            df["tagged"].append(False)
            found_term1 = False
            found_term2 = False
            found_sentences = []
            for sentence in bio_sentences:
                sentence = str(sentence)
                if terms[0] in sentence:
                    found_term1 = True
                if terms[1] in sentence:
                    found_term2 = True
                if terms[0] in sentence and terms[1] in sentence:
                    found_sentences.append(sentence)
            sentences = found_sentences
            df["term1_found"].append(found_term1)
            df["term2_found"].append(found_term2)
            df["textbook_match"].append(len(found_sentences) > 0)
                
        df["count_sentences"].append(count_sentences)
        df["sentences"].append("\n".join(sentences))
                    
for cp in df["concept-pair"]:
    df["synonym-tagged"].append(cp in found_concepts)

df_df = pd.DataFrame(df)
df_df.to_excel(f"{data_dir}/summary/relations_classifications.xlsx")
df_df.head()

no-relation
subclass-of
Processing term pair 0/19179
Processing term pair 100/19179
Processing term pair 200/19179
Processing term pair 300/19179
Processing term pair 400/19179
Processing term pair 500/19179
Processing term pair 600/19179
Processing term pair 700/19179
Processing term pair 800/19179
Processing term pair 900/19179
Processing term pair 1000/19179
Processing term pair 1100/19179
Processing term pair 1200/19179
Processing term pair 1300/19179
Processing term pair 1400/19179
Processing term pair 1500/19179
Processing term pair 1600/19179
Processing term pair 1700/19179
Processing term pair 1800/19179
Processing term pair 1900/19179
Processing term pair 2000/19179
Processing term pair 2100/19179
Processing term pair 2200/19179
Processing term pair 2300/19179
Processing term pair 2400/19179
Processing term pair 2500/19179
Processing term pair 2600/19179
Processing term pair 2700/19179
Processing term pair 2800/19179
Processing term pair 2900/19179
Processing term pair 3000/19

,sentences,relation,term-pair,concept-pair,tagged,synonym-tagged,count_sentences,textbook_match,term1_found,term2_found
0,<e1> Chemical </e1> reactions alter the atomic...,subclass-of,chemical -> substance,Chemical -> Substance,True,True,17,True,True,True
1,<e1> Chemical </e1> reactions obey the laws of...,subclass-of,chemical -> matter,Chemical -> Substance,True,True,5,True,True,True
2,,subclass-of,chemical -> object,Chemical-Entity -> Physical-Object,False,False,0,False,True,True
3,,subclass-of,substance -> object,Substance -> Tangible-Entity,False,False,0,False,True,True
4,,subclass-of,chemical entity -> object,Chemical-Entity -> Physical-Object,False,False,0,False,False,True


In [12]:
def group_pairs(row):
    if row["tagged"]:
        group = "Tagged"
    elif row["textbook_match"]:
        group = "TB_Match_Not_Tagged"
    elif row["synonym-tagged"]:
        group = "Synonym Tagged"
    elif not row["term1_found"] or not row["term2_found"]:
        group = "Missing Term"
    else:
        group = "Terms Not Same Sentence"
    return group

df_copy = df_df.copy()
df_copy["group"] = df_copy.apply(group_pairs, axis=1)
df_copy.head()

,sentences,relation,term-pair,concept-pair,tagged,synonym-tagged,count_sentences,textbook_match,term1_found,term2_found,group
0,<e1> Chemical </e1> reactions alter the atomic...,subclass-of,chemical -> substance,Chemical -> Substance,True,True,17,True,True,True,Tagged
1,<e1> Chemical </e1> reactions obey the laws of...,subclass-of,chemical -> matter,Chemical -> Substance,True,True,5,True,True,True,Tagged
2,,subclass-of,chemical -> object,Chemical-Entity -> Physical-Object,False,False,0,False,True,True,Terms Not Same Sentence
3,,subclass-of,substance -> object,Substance -> Tangible-Entity,False,False,0,False,True,True,Terms Not Same Sentence
4,,subclass-of,chemical entity -> object,Chemical-Entity -> Physical-Object,False,False,0,False,False,True,Missing Term


In [13]:
df_copy.groupby(["relation", "group"])["sentences"].count().reset_index().rename({"sentences": "count"}, axis=1).to_csv(f"{data_dir}/summary/relations_matches_summary.csv", index=False)